In [2]:
# Import the required packages
using JuMP
using CSV
using DataFrames
using PowerModels
using Ipopt
using Juniper

In [3]:
#Read in data
baseMVA = 10;

bus_data = CSV.read("bus_data.csv", DataFrame) |> Matrix;
(num_busses, var) = size(bus_data);
VMax = bus_data[:, 4];
VMin = bus_data[:, 7]; 
load_data = CSV.read("load_data.csv", DataFrame) |> Matrix;
(num_loads, var) = size(load_data);
branch_data = CSV.read("branch_data.csv", DataFrame) |> Matrix;
(num_branches, var) = size(branch_data)
G = CSV.read("G_mat.csv", DataFrame) |> Matrix;
B = CSV.read("B_mat.csv", DataFrame)|> Matrix;

#Expand the Pd data such that non load busses are given power demands of 0
Pd = load_data[:,5]*baseMVA
load_busses = load_data[:,2];
bus_idx = bus_data[:,2]; 
Pd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(bus_idx[i]), load_busses)
    if !isempty(idx)
        Pd_mod[i] = Pd[idx[1]]
    else
        Pd_mod[i] = 0;
    end
end
Pd = Pd_mod

Qd = load_data[:,4]*baseMVA
Qd_mod = zeros(1, num_busses);
for i = 1:num_busses
    idx = findall(isequal(bus_idx[i]), load_busses)
    if !isempty(idx)
        Qd_mod[i] = Qd[idx[1]]
    else
        Qd_mod[i] = 0;
    end
end
Qd = Qd_mod

1×18 Matrix{Float64}:
 0.0  0.12  0.25  0.93  2.26  0.5  0.12  …  0.5  0.31  0.62  0.12  0.0  0.0

In [6]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses])
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)
@variable(model, I[1:num_busses, 1:num_busses] >= 0)
@variable(model, VP >= 0)

M = 10000;

@constraint(model, [i in 1:num_busses], y[i] <= M*x[i])
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))
#This case has no flow limits
#@NLconstraint(model, [i in 1:num_busses, j in 1:num_busses], I[i,j] == sqrt(G[i,j]^2+B[i,j]^2)*sqrt(V[i]^2+V[j]^2-2*V[i]*V[j]*cos(theta[j]-theta[i])))
@NLconstraint(model, [i in 1:num_busses], (PG[i]^2+QG[i]^2)^(1/2) <= y[i])
#@constraint(model, [i in 1:num_busses], QG[i] <= y[i])
#This case has no flow limits
#@constraint(model, [i in 1:num_busses, j in 1:num_busses] I[i,j] <= Imax[i,j])
@constraint(model, [i in 1:num_busses-1], VMin[i] <= V[i] <= VMax[i])
@constraint(model, V[num_busses] == 0)
@constraint(model, theta[num_busses] == 0)
@constraint(model, VP == sum(V[i]*Pd[i] for i in 1:num_busses))
@constraint(model, [i in 1:num_busses],  -90 <= theta[i] <= 90 )


c_fix = 0;

c_var = 1;
beta = .5;
gamma = 50;

@objective(model, Min, sum(c_fix*x[i]+c_var*y[i] for i in 1:num_busses) +
                        beta*VP +
                        gamma*sum(Ploss[i] for i in 1:num_busses))

optimize!(model)


nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 469
#IntBinVar: 18
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.40799999237060547
Relaxation Obj: 250.4089777468306

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  


┌ Warning: Only almost solved
└ @ Juniper C:\Users\riley\.julia\packages\Juniper\HBPrQ\src\BnBTree.jl:116


    2       2                 -                         250.41              -     29.8      5         -     
    3       3                 -                         250.41              -     30.1      -       77.7%   
    4       4                 -                         250.41              -     30.4      -       77.7%   
    5       5                 -                         250.41              -     30.8      -       77.7%   
    6       6                 -                         250.41              -     31.2      -       77.7%   
    7       7                 -                         250.41              -     31.5      -       77.7%   
    8       8                 -                         250.41              -     32.2      -       77.7%   
    9       9                 -                         250.41              -     32.5      -       77.7%   
   10       10                -                         250.41              -     32.9      -       77.7%   
   11       11     

In [11]:
optimizer = Juniper.Optimizer
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver))

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses])
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)
@variable(model, I[1:num_busses, 1:num_busses] >= 0)
@variable(model, VP >= 0)

M = 10000;

#In bus data, a 3 indicates the slack bus, the connection to the grid
slack_idx = findall(isequal(3), bus_data[:,3])
non_slack_num = filter(x->x!=slack_idx, 1:num_busses)

@constraint(model, [i in 1:num_busses], y[i] <= M*x[i])
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@constraint(model, [i in 1:num_busses], Ploss[i] <= Pd[i])
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))
#This case has no flow limits
#@NLconstraint(model, [i in 1:num_busses, j in 1:num_busses], I[i,j] == sqrt(G[i,j]^2+B[i,j]^2)*sqrt(V[i]^2+V[j]^2-2*V[i]*V[j]*cos(theta[j]-theta[i])))
#When using capacity constraint sqrt(Pg^2+Qg^2) <= y, becomes infeasible
#@constraint(model, [i in 1:num_busses], PG[i]+QG[i] <= y[i])
@NLconstraint(model, [i in 1:num_busses], (PG[i]^2+QG[i]^2)^(1/2) <= y[i])
#This case has no flow limits
#@constraint(model, [i in 1:num_busses, j in 1:num_busses] I[i,j] <= Imax[i,j])
@constraint(model, [i in non_slack_num], VMin[i] <= V[i] <= VMax[i])
#@constraint(model, V[1] == 0)
#@constraint(model, theta[1] == 0)
@constraint(model, VP == sum(V[i]*Pd[i] for i in 1:num_busses))
@constraint(model, [i in 1:num_busses],  -pi/2 <= theta[i] <= pi/2 )

#Varying these parameters changes output behavior
#When gamma < 1, tends to just shed all load, bad!
#For gamma = 1, and slightly above sheds some load, for gamma well above 1, like 2, 
#Serves all load. Interesting! Hyper parameter tuning 
c_fix = 0;
c_var = 1;
beta = .5;
gamma = 5;

@objective(model, Min, sum(c_fix*x[i]+c_var*y[i] for i in 1:num_busses) +
                        beta*VP +
                        gamma*sum(Ploss[i] for i in 1:num_busses))

optimize!(model)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 469
#IntBinVar: 18
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.3269999027252197
Relaxation Obj: 18.921877581033534

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                         18.92               -     2.3       0         -     
    3       3                 -                         18.92               -     2.4       -       77.7%   
    4       4                 -                         18.92               -     2.6       -       77.7%   
    5       5                 -                         18.92               -     2.7       -       77.7% 

In [17]:
JuMP.value.(PG)

18-element Vector{Float64}:
 0.0005764805234258504
 0.1589915138740683
 0.32717449563029755
 1.230317079675314
 2.68439514294148
 0.8454173131183804
 0.17728274977624783
 1.0220822757582106
 0.40382572197265065
 0.8157852899421068
 0.24924958708323686
 0.16367115584525302
 0.6702756304646617
 0.41983427648435057
 0.955828899156598
 0.20931806830092364
 0.000913995478579609
 1.316948657382304

In [ ]:
#optimizer = Juniper.Optimizer
#nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

model = Model(Ipopt.Optimizer)

@variable(model, V[1:num_busses] >= 0)
@variable(model, theta[1:num_busses])
@variable(model, PG[1:num_busses] >= 0)
@variable(model, QG[1:num_busses] >= 0)
@variable(model, y[1:num_busses] >= 0)
#@variable(model, x[1:num_busses], Bin)
@variable(model, Ploss[1:num_busses] >= 0)
@variable(model, Qloss[1:num_busses] >= 0)
@variable(model, I[1:num_busses, 1:num_busses] >= 0)
@variable(model, VP >= 0)

M = 10000;

#In bus data, a 3 indicates the slack bus, the connection to the grid
slack_idx = findall(isequal(3), bus_data[:,3])
non_slack_num = filter(x->x!=slack_idx, 1:num_busses)

@constraint(model, [i in 1:num_busses], y[i] <= M)
@NLconstraint(model, [i in 1:num_busses], PG[i] - Pd[i] + Ploss[i] == sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
@constraint(model, [i in 1:num_busses], Ploss[i] <= Pd[i])
@NLconstraint(model, [i in 1:num_busses], QG[i] - Qd[i] == sum((V[i]*V[j])*(G[i,j]*sin(theta[i]-theta[j])-B[i,j]*cos(theta[i]-theta[j])) for j in 1:num_busses))
#This case has no flow limits
#@NLconstraint(model, [i in 1:num_busses, j in 1:num_busses], I[i,j] == sqrt(G[i,j]^2+B[i,j]^2)*sqrt(V[i]^2+V[j]^2-2*V[i]*V[j]*cos(theta[j]-theta[i])))
#When using capacity constraint sqrt(Pg^2+Qg^2) <= y, becomes infeasible
#@constraint(model, [i in 1:num_busses], PG[i]+QG[i] <= y[i])
@NLconstraint(model, [i in 1:num_busses], (PG[i]^2+QG[i]^2)^(1/2) <= y[i])
#This case has no flow limits
#@constraint(model, [i in 1:num_busses, j in 1:num_busses] I[i,j] <= Imax[i,j])
@constraint(model, [i in non_slack_num], VMin[i] <= V[i] <= VMax[i])
#@constraint(model, V[1] == 0)
#@constraint(model, theta[1] == 0)
@constraint(model, VP == sum(V[i]*Pd[i] for i in 1:num_busses))
@constraint(model, [i in 1:num_busses],  -pi/2 <= theta[i] <= pi/2 )

#Varying these parameters changes output behavior
#When gamma < 1, tends to just shed all load, bad!
#For gamma = 1, and slightly above sheds some load, for gamma well above 1, like 2, 
#Serves all load. Interesting! Hyper parameter tuning 
c_fix = 0;
#NREL's SAM DATA default costs for photovoltaic residential setting
c_var = 2.76;
beta = .5;
gamma = 5;

@objective(model, Min, sum(c_var*y[i] for i in 1:num_busses) +
                        beta*VP +
                        gamma*sum(Ploss[i] for i in 1:num_busses))

optimize!(model)

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     1367
Number of nonzeros in inequality constraint Jacobian.:      125
Number of nonzeros in Lagrangian Hessian.............:     4446

Total number of variables............................:      451
                     variables with only lower bounds:      433
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       38
Total number of inequality constraints...............:       89
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       35
        inequality constraints with only upper bounds:       54

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0849989e+00 2.98e+00 1.66e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [95]:
V = JuMP.value.(V)
theta = JuMP.value.(theta)
i = 17
println(sum((V[i]*V[j])*(G[i,j]*cos(theta[i]-theta[j])+B[i,j]*sin(theta[i]-theta[j])) for j in 1:num_busses))
println(sum(JuMP.value.(PG)))
println(sum(Pd))


34.10291728708915
41.33766404557758
7.6
